# Use of the Slack Api to fetch the history of the channel

In [31]:
import requests 
import os
from dotenv import load_dotenv
import pandas as pd
import nltk
import ast
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity as distance
from src.app import app
from pymongo import MongoClient
from src.helpers.errorHandler import errorHandler ,Error404 ,APIError
from src.config import DBURL
from bson.json_util import dumps
from bson import ObjectId
import json
load_dotenv()

# Load the apikey
token = os.getenv("TOKENSLACK")
channel=os.getenv('CHANNEL')
print("WE HAVE TOKEN") if token else print("NO TOKEN FOUND")
print("WE HAVE CHANNEL") if channel else print("NO CHANNEL FOUND")


WE HAVE TOKEN
WE HAVE CHANNEL


In [4]:
url='https://slack.com/api/channels.history?token={token}&channel={channel}&count=678&pretty=1'


In [10]:
from slacker import Slacker
# Authenticate with slacker
slack = Slacker(token)

In [11]:
# Check for success
if slack.api.test().successful:
    print(
        f"Connected to {slack.team.info().body['team']['name']}.")
else:
    print('Try Again!')

Connected to Data Mad March 2020.


In [18]:
r = slack.channels.history( channel=channel,count=678)

history = r.body

In [25]:
#Retrive all the text and user
chat=[]
for mes in history['messages']:
    try:
        chat.append({
            'user': mes['user'],
            'text': mes['text']
        })
    except:
        continue

In [27]:
df=pd.DataFrame(chat)

In [30]:
display(df.head())

,user,text
0,U010F1FCBGU,gracias tío!!!
1,U010H826R1D,"Chicos, ya tenéis la semana 3 de clases en mi ..."
2,U010F1FCBGU,<https://www.aprendemachinelearning.com/ejerci...
3,U010F1FCBGU,Buenas! Dejo esto por aquí
4,U0102M41WV8,muchas gracias david!!


# Adding of the datamad0320 chat to the database

In [ ]:
#set of datamad0320 users
users_unique=set(df['user'].values)

In [90]:
#add the users to mongodb database
usersIds={}
for username in users_unique:
    r = requests.get(f'http://localhost:3500///user/create/{username}')
    usersIds[username]=(r.json()['user_id'])


In [94]:
list(usersIds.values())[0:5]

['5eaec9762908c7e2052a6ab8',
 '5eaec9762908c7e2052a6ab9',
 '5eaec9762908c7e2052a6aba',
 '5eaec9762908c7e2052a6abb',
 '5eaec9762908c7e2052a6abc']

In [95]:
#creation of the new chat dataMad0320
chat_name='dataMad0320'
users_ids=list(usersIds.values())
r = requests.get(f'http://localhost:3500//chat/create?ids={users_ids}&name={chat_name}')
chatId=r.json()['chat_id']

In [96]:
chatId

'5eaec9f02908c7e2052a6ad0'

In [97]:
#add the messages to the chat
messagesIds=[]
for mes in chat:#chat is a list of dictionaties {'user':user,'text':text}
    user=mes['user']
    user_id=usersIds[user]
    text=mes['text']
    r = requests.get(f'http://localhost:3500///chat/{chatId}/addmessage?user_id={user_id}&text={text}')
    messagesIds.append(r.json()['message_id'])

# Sentiment analysis of the chat

In [99]:
from classifier import *
clf = SentimentClassifier()
clf.predict(e)
r = requests.get(f'http://localhost:3500///chat/{chatId}/sentiment?lang=es')
r.json()['chat_sentiment']


/Users/silviaserafini/Library/Python/3.6/lib/python/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


-0.4168880100205129